# Práctica 6: Sistema multiagente

**S**: Suceptible      **I**: Infectado      **R**: Recuperado

### Variación para la Tarea

In [4]:
l <- 1.5
n <- 50
pi <- 0.05
pr <- 0.02
pv <- 0.1
v <- l / 30

contagios <- function(){
agentes <- data.frame(x = double(), y = double(),
                      dx = double(), dy = double(),
                      estado  = character())
for (i in 1:n) {
    e <- "S"
    
    if (runif(1) < pv) {
        e <- "V"
    }
    
    if (runif(1) < pi && e!="V") {
        e <- "I"
    }
    agentes <- rbind(agentes, data.frame(x = runif(1, 0, l),
                                         y = runif(1, 0, l),
                                         dx = runif(1, -v, v),
                                         dy = runif(1, -v, v),
                                         estado = e))
 
}
levels(agentes$estado) <- c("S", "I", "R", "V")
epidemia <- integer()
r <- 0.1
tmax <- 100
#digitos <- floor(log(tmax, 10)) + 1
for (tiempo in 1:tmax) {
    infectados <- dim(agentes[agentes$estado == "I",])[1]
    epidemia <- c(epidemia, infectados)
    if (infectados == 0) {
        break
    }
    contagios <- rep(FALSE, n)
    for (i in 1:n) { # posibles contagios
        a1 <- agentes[i, ]
        if (a1$estado == "I") { # desde los infectados
            for (j in 1:n) {
                if (!contagios[j]) { # aun sin contagio
                    a2 <- agentes[j, ]
                    if (a2$estado == "S") { # hacia los susceptibles
                        dx <- a1$x - a2$x
                        dy <- a1$y - a2$y
                        d <- sqrt(dx^2 + dy^2)
                        if (d < r) { # umbral
                            p <- (r - d) / r
                            if (runif(1) < p) {
                                contagios[j] <- TRUE
                            }
                        }
                    }
                }
            }
        }
    }
    for (i in 1:n) { # movimientos y actualizaciones
        a <- agentes[i, ]
        if (contagios[i]) {
            a$estado <- "I"
        } else if (a$estado == "I") { # ya estaba infectado
            if (runif(1) < pr) {
                a$estado <- "R" # recupera
            }
        }
        a$x <- a$x + a$dx
        a$y <- a$y + a$dy
        if (a$x > l) {
            a$x <- a$x - l
        }
        if (a$y > l) {
            a$y <- a$y - l
        }
        if (a$x < 0) {
            a$x <- a$x + l
        }
        if (a$y < 0) {
            a$y <- a$y + l
        }
        agentes[i, ] <- a
    }
}
return(epidemia)
}

In [5]:

datos=data.frame()
for(pv in seq(0,1,0.05)){ 
    for (i in 1:100){
        epi <- contagios()
        datos <- rbind(datos, c(i, pv, max(epi*100/n), which.max(epi)))
    }
}


In [21]:
print(xtable::xtable(datos[c(1,120,502,902,2005),]))

% latex table generated in R 4.0.2 by xtable 1.8-4 package
% Tue Oct 27 17:55:32 2020
\begin{table}[ht]
\centering
\begin{tabular}{rrrrr}
  \hline
 & Iteración & PV & Porcentaje & MAX ITER \\ 
  \hline
1 & 1.00 & 0.00 & 62.00 & 48.00 \\ 
  120 & 20.00 & 0.05 & 58.00 & 51.00 \\ 
  502 & 2.00 & 0.25 & 48.00 & 57.00 \\ 
  902 & 2.00 & 0.45 & 24.00 & 35.00 \\ 
  2005 & 5.00 & 1.00 & 0.00 & 1.00 \\ 
   \hline
\end{tabular}
\end{table}


In [8]:
names(datos)=c("Iteración","PV","Porcentaje", "MAX ITER")

In [11]:
png(file="max_iter.png",
    width=5700, height=2000, res = 200)
par(mar = c(5,5,5,5))
boxplot(datos[1:100,4], datos[101:200,4], datos[201:300,4], datos[301:400,4], datos[401:500,4],
        datos[501:600,4], datos[601:700,4], datos[701:800,4], datos[801:900,4], datos[901:1000,4], datos[1001:1100,4],
        datos[1101:1200,4], datos[1201:1300,4], datos[1301:1400,4], datos[1401:1500,4], datos[1501:1600,4], 
        datos[1601:1700,4], datos[1701:1800,4], datos[1801:1900,4], datos[1901:2000,4], datos[2001:2100,4],
        names=seq(0,1,0.05), col=palette("Pastel 2"), ylab="Iteración donde se alcanzó el máximo porcentaje de contagiados", xlab="Probabilidad de vacuna",
       cex.axis=1.6, cex.lab=2)
dev.off()

pdf 
  2

In [87]:
png(file="boxplot1.png",
    width=5700, height=2000, res = 200)
par(mar = c(5,5,5,5))
boxplot(datos[1:100,3], datos[101:200,3], datos[201:300,3], datos[301:400,3], datos[401:500,3],
        datos[501:600,3], datos[601:700,3], datos[701:800,3], datos[801:900,3], datos[901:1000,3], datos[1001:1100,3],
        datos[1101:1200,3], datos[1201:1300,3], datos[1301:1400,3], datos[1401:1500,3], datos[1501:1600,3], 
        datos[1601:1700,3], datos[1701:1800,3], datos[1801:1900,3], datos[1901:2000,3], datos[2001:2100,3],
        names=seq(0,1,0.05), col=palette("Pastel 2"), ylab="Porcentaje máximo de contagiados", xlab="Probabilidad de vacuna",
       cex.axis=1.6, cex.lab=2)
dev.off()

pdf 
  2

In [10]:
cor.test(datos$PV, datos$Porcentaje)


	Pearson's product-moment correlation

data:  datos$PV and datos$Porcentaje
t = -55.98, df = 2098, p-value < 2.2e-16
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 -0.7905482 -0.7562048
sample estimates:
      cor 
-0.773945 


## Reto 1:

In [103]:
l <- 1.5
n <- 50
pi <- 0.05
pr <- 0.02
pv <- 0.1

contagios_1 <- function(){
agentes <- data.frame(x = double(), y = double(),
                      xmeta =double(), ymeta=double(),
                      v=double(),
                      estado  = character())
for (i in 1:n) {
    e <- "S"
    
    if (runif(1) < pv) {
        e <- "V"
    }
    
    if (runif(1) < pi && e!="V") {
        e <- "I"
    }
    agentes <- rbind(agentes, data.frame(x = runif(1, 0, l),
                                         y = runif(1, 0, l),
                                         xmeta = runif(1,0,l),
                                         ymeta = runif(1,0,l),
                                         estado = e))
    }
    dx <- c()
    dy <- c()
    for (i in 1:50){
        dx[i] <- abs(agentes$x[i] - agentes$xmeta[i]) /10
        dy[i] <- abs(agentes$y[i] - agentes$ymeta[i]) /10
    }
  agentes <- cbind(agentes,dx)
  agentes <- cbind(agentes,dy)

 

levels(agentes$estado) <- c("S", "I", "R", "V")
epidemia <- integer()
r <- 0.1
tmax <- 100
digitos <- floor(log(tmax, 10)) + 1
for (tiempo in 1:tmax) {
    infectados <- dim(agentes[agentes$estado == "I",])[1]
    epidemia <- c(epidemia, infectados)
    if (infectados == 0) {
        break
    }
    contagios <- rep(FALSE, n)
    for (i in 1:n) { # posibles contagios
        a1 <- agentes[i, ]
        if (a1$estado == "I") { # desde los infectados
            for (j in 1:n) {
                if (!contagios[j]) { # aun sin contagio
                    a2 <- agentes[j, ]
                    if (a2$estado == "S") { # hacia los susceptibles
                        dx <- a1$x - a2$x
                        dy <- a1$y - a2$y
                        d <- sqrt(dx^2 + dy^2)
                        if (d < r) { # umbral
                            p <- (r - d) / r
                            if (runif(1) < p) {
                                contagios[j] <- TRUE
                            }
                        }
                    }
                }
            }
        }
    }
    for (i in 1:n) { # movimientos y actualizaciones
        a <- agentes[i, ]
        if (contagios[i]) {
            a$estado <- "I"
        } else if (a$estado == "I") { # ya estaba infectado
            if (runif(1) < pr) {
                a$estado <- "R" # recupera
            }
        }
        
    if (round(a$x,7)==round(a$xmeta,7) && round(a$y,7)==round(a$ymeta,7)){
            a$xmeta <-runif(1,0,l)
            a$ymeta <- runif(1,0,l)
            a$dx <- abs(a$x-a$xmeta)/10
            a$dy <- abs(a$y-a$ymeta)/10
        
            
            if(a$x < a$xmeta){a$x <- a$x + a$dx}
            else{a$x<-a$x-a$dx}
        
            if(a$y<a$ymeta){a$y <- a$y + a$dy}
            else{a$y<-a$y-a$dy}
            
            }
    else{
        if(a$x < a$xmeta){a$x <- a$x + a$dx}
        else{a$x<-a$x-a$dx}
        
        if(a$y<a$ymeta){a$y <- a$y + a$dy}
        else{a$y<-a$y-a$dy}
        }
        agentes[i, ] <- a
        }
    
    aS <- agentes[agentes$estado == "S",]
    aI <- agentes[agentes$estado == "I",]
    aR <- agentes[agentes$estado == "R",]
    aV <- agentes[agentes$estado == "V",]
    tl <- paste(tiempo, "", sep="")
    while (nchar(tl) < digitos) {
        tl <- paste("0", tl, sep="")
    }
    salida <- paste("p6_t", tl, ".png", sep="")
    tiempo <- paste("Paso", tiempo)
    png(salida)
    plot(l, type="n", main=tiempo, xlim=c(0, l), ylim=c(0, l), xlab="x", ylab="y")
    if (dim(aS)[1] > 0) {
        points(aS$x, aS$y, pch=15, col="chartreuse3", bg="chartreuse3")
    }
    if (dim(aI)[1] > 0) {
        points(aI$x, aI$y, pch=16, col="firebrick2", bg="firebrick2")
    }
    if (dim(aR)[1] > 0) {
        points(aR$x, aR$y, pch=17, col="goldenrod", bg="goldenrod")
    }
    if (dim(aV)[1] > 0) {
        points(aV$x, aV$y, pch=18, col="lightseagreen", bg="lightseagreen")
    }
    graphics.off()
}


return(epidemia)
}

In [53]:

for(pv in seq(0,1,0.05)){
    for (i in 1:100){
        epi <- contagios_1()
        datos_1 <- rbind(datos_1, c(i, pv, max(epi*100/n), which.max(epi)))
    }
}


In [54]:

for(pv in seq(0.15,0.95,0.1)){
    for (i in 1:100){
        epi <- contagios_1()
        datos_1 <- rbind(datos_1, c(i, pv, max(epi*100/n), which.max(epi)))
    }
}


In [86]:
png(file="boxplot_reto1.png",
    width=5700, height=2000, res = 200)
par(mar = c(5,5,5,5))
boxplot(datos_1[1:100,3], datos_1[101:200,3], datos_1[201:300,3], datos_1[1201:1300,3], datos_1[301:400,3], datos_1[1301:1400,3],
        datos_1[401:500,3], datos_1[1401:1500,3], datos_1[501:600,3], datos_1[1501:1600,3], datos_1[601:700,3], datos_1[1601:1700,3],
        datos_1[701:800,3], datos_1[1701:1800,3], datos_1[801:900,3], datos_1[1801:1900,3], datos_1[901:1000,3], datos_1[1901:2000,3],
        datos_1[1001:1100,3], datos_1[2001:2100,3], datos_1[1101:1200,3],     
        names=seq(0,1,0.05), col=palette("Pastel 2"), ylab="Porcentaje máximo de contagiados", xlab="Probabilidad de vacuna",
       cex.axis=1.6, cex.lab=2)
dev.off()

pdf 
  2

In [89]:
png(file="maxiter_reto1.png",
    width=5700, height=2000, res = 200)
par(mar = c(5,5,5,5))
boxplot(datos_1[1:100,4], datos_1[101:200,4], datos_1[201:300,4], datos_1[1201:1300,4], datos_1[301:400,4], datos_1[1301:1400,4],
        datos_1[401:500,4], datos_1[1401:1500,4], datos_1[501:600,4], datos_1[1501:1600,4], datos_1[601:700,4], datos_1[1601:1700,4],
        datos_1[701:800,4], datos_1[1701:1800,4], datos_1[801:900,4], datos_1[1801:1900,4], datos_1[901:1000,4], datos_1[1901:2000,4],
        datos_1[1001:1100,4], datos_1[2001:2100,4], datos_1[1101:1200,4],     
        names=seq(0,1,0.05), col=palette("Pastel 2"), ylab="Iteración donde se alcanzó el máximo porcentaje de contagiados", xlab="Probabilidad de vacuna",
       cex.axis=1.6, cex.lab=2)
dev.off()

pdf 
  2

In [91]:
names(datos_1)=c("Iteración","PV","Porcentaje", "MAX ITER")

In [92]:
cor.test(datos_1$PV, datos_1$Porcentaje)


	Pearson's product-moment correlation

data:  datos_1$PV and datos_1$Porcentaje
t = -54.801, df = 2098, p-value < 2.2e-16
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 -0.7843157 -0.7490935
sample estimates:
       cor 
-0.7672826 
